In [1]:
# DATA PREPARATION NOTEBOOK
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
# Load messy dataset
df = pd.read_csv("messy_data.csv")  # replace with your file path
df.head()

,EmployeeID,Name,Department,PositionTitle,Salary,HireDate,Benefits_Cost
0,1001,John Smith,Police,Police Officer I,55000,2015-01-10,12500
1,1002,Jane Doe,Fire,Firefighter,60000.5,2014/03/15,14000
2,1003,Peter Jones,Parks & Rec,Parks Maintenance,52000,2018-05-22,11000
3,1004,Sarah Lee,Public Works,Sanitation Worker,48000.75,2019-11-01,10500
4,1005,NaN,Finance,Accountant I,65000,2016-07-20,15000


In [4]:
# Inspect dataset
df.info()        # column types and missing values
df.describe()    # summary statistics
df.isnull().sum()  # count missing values per column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   EmployeeID     15 non-null     int64 
 1   Name           14 non-null     object
 2   Department     15 non-null     object
 3   PositionTitle  14 non-null     object
 4   Salary         15 non-null     object
 5   HireDate       15 non-null     object
 6   Benefits_Cost  15 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 972.0+ bytes


EmployeeID       0
Name             1
Department       0
PositionTitle    1
Salary           0
HireDate         0
Benefits_Cost    0
dtype: int64

In [5]:
# Fill missing numeric values with median
num_cols = df.select_dtypes(include=np.number).columns
for col in num_cols:
    df[col].fillna(df[col].median(), inplace=True)

# Fill missing categorical values with mode
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

C:\Users\Shafia\AppData\Local\Temp\ipykernel_26140\2365776739.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\Shafia\AppData\Local\Temp\ipykernel_26140\2365776739.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [6]:
# Remove outliers using Z-score
numeric_df = df[num_cols]
z_scores = np.abs(stats.zscore(numeric_df))
df = df[(z_scores < 3).all(axis=1)]

In [7]:
# Convert columns to appropriate types
if 'date_col' in df.columns:
    df['date_col'] = pd.to_datetime(df['date_col'])

if 'category_col' in df.columns:
    df['category_col'] = df['category_col'].astype('category')

In [8]:
# Save cleaned dataset for modeling
df.to_csv("cleaned_data.csv", index=False)
print("Data cleaning completed! Cleaned dataset saved as 'cleaned_data.csv'.")

Data cleaning completed! Cleaned dataset saved as 'cleaned_data.csv'.


In [9]:
# Pivot data if needed: example of summarizing values by category over time
if 'date_col' in df.columns and 'category_col' in df.columns and 'value' in df.columns:
    df_pivot = df.pivot_table(
        index='date_col',          # rows by date
        columns='category_col',    # columns by category
        values='value',            # aggregate value
        aggfunc='sum'              # sum values per group
    )
    df_pivot.head()

In [10]:
# Create a multi-index (example: region and year)
if 'region' in df.columns and 'year' in df.columns:
    df.set_index(['region', 'year'], inplace=True)
    df.sort_index(inplace=True)
    df.head()

In [11]:
# Merge datasets using merge_asof for time-aligned joins
# Example: merging df1 and df2 on 'date_col'
# df1 and df2 must be sorted by date
if 'df1' in globals() and 'df2' in globals() and 'date_col' in df1.columns:
    df_merged = pd.merge_asof(
        df1.sort_values('date_col'),
        df2.sort_values('date_col'),
        on='date_col',
        direction='nearest'  # nearest date match
    )
    df_merged.head()

In [12]:
# Save normalized dataset
df.to_csv("normalized_data.csv", index=True)  # keep multi-index if applied
print("Normalized dataset saved as 'normalized_data.csv'.")

Normalized dataset saved as 'normalized_data.csv'.


In [13]:
# Example: creating a squared term for a numeric feature
if 'feature' in df.columns:
    df['feature_sq'] = df['feature'] ** 2
    df[['feature', 'feature_sq']].head()

In [14]:
# Example: interaction between two numeric features
if 'feature1' in df.columns and 'feature2' in df.columns:
    df['interaction'] = df['feature1'] * df['feature2']
    df[['feature1', 'feature2', 'interaction']].head()

In [15]:
# Convert categorical columns to dummy variables (one-hot encoding)
cat_cols = df.select_dtypes(include='category').columns
if len(cat_cols) > 0:
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    df.head()

In [16]:
# Example: mean of target per group
if 'group_col' in df.columns and 'target' in df.columns:
    df['mean_by_group'] = df.groupby('group_col')['target'].transform('mean')
    df[['group_col', 'target', 'mean_by_group']].head()

In [17]:
# Save the fully prepared dataset
df.to_csv("prepared_data.csv", index=False)
print("Feature engineering completed! Dataset saved as 'prepared_data.csv'.")

Feature engineering completed! Dataset saved as 'prepared_data.csv'.
